<a href="https://colab.research.google.com/github/dnevo/Practicum/blob/master/S02_Data_Preprocessing_%E2%80%93_correlate_bank_customer_params_to_credit_score.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Comments from reviewer

<span style="color:green"> Hi! Congratulations on your first project :) I know that the first project is the hardest one, but you did a good job. I believe we can make your work even better. I left you comments on what should be improved and also questions I'd like to get answers to. You will find my comments in Markdown cells with green text. Please, change your work a bit and send the project back to me. Good Luck! :) </span>

## Comments from reviewer 2

<span style="color:purple">Thanks for the udpdated project!</span>

## Analyzing borrowers’ risk of defaulting

Your project is to prepare a report for a bank’s loan division. You’ll need to find out if a customer’s marital status and number of children has an impact on whether they will default on a loan. The bank already has some data on customers’ credit worthiness.

Your report will be considered when building a **credit scoring** of a potential customer. A ** credit scoring ** is used to evaluate the ability of a potential borrower to repay their loan.

### Step 1. Open the data file and have a look at the general information. 

In [ ]:
import pandas as pd
import numpy as np
pd.set_option('display.width', 200)
credit_scoring = pd.read_csv('/datasets/credit_scoring_eng.csv')
print(credit_scoring.tail(30))

       children  days_employed  dob_years            education  education_id      family_status  family_status_id gender    income_type  debt   total_income  \
21495         1            NaN         50  secondary education             1  civil partnership                 1      F       employee     0            NaN   
21496         1    -759.680851         31  secondary education             1            married                 0      F       employee     0  119392.620079   
21497         0            NaN         48       MASTERS DEGREE             0            married                 0      F        partner     0            NaN   
21498         1   -1330.627998         32  secondary education             1  civil partnership                 1      M       employee     0  240767.575887   
21499         0   -9929.015065         57  secondary education             1  civil partnership                 1      M        partner     0  157553.158589   
21500         0    -578.082757         2

In [ ]:
print(credit_scoring.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


#### it can be noticed that the data is not distributed evenly according to the `debt` column - it contains only 1741 records of people who returned the loan - i.e. only 8% of the records.

In [ ]:
print(credit_scoring['debt'].value_counts())

0    19784
1     1741
Name: debt, dtype: int64


### Conclusion

#### main point I notice is that only 8% of the records are of people who did not returned loan.
Two column `days_employed` and `total_income` were identified as containing missing data.

### Step 2. Data preprocessing

#### invalid data: invalid `gender` (=`XNA`). Replace by Female (`F`)
It was was replaced to F as Female were the majority (67%) in the data

<span style="color:green"> Why would you change it to F? Can you justify your decision? </span>

In [ ]:
print(credit_scoring['gender'].value_counts())
credit_scoring['gender'] = credit_scoring['gender'].where(credit_scoring['gender'] != 'XNA', other='F')

F      14236
M       7288
XNA        1
Name: gender, dtype: int64


#### invalid data: invalid age (`dob_years` =`0`). Replace by the mean.

In [ ]:
print(credit_scoring['dob_years'].value_counts().sort_index().head())
mean_age = int(credit_scoring['dob_years'].mean())
print('Mean age: {}'.format(mean_age))
credit_scoring['dob_years'] = credit_scoring['dob_years'].where(credit_scoring['dob_years']>0, other=mean_age)

0     101
19     14
20     51
21    111
22    183
Name: dob_years, dtype: int64
Mean age: 43


#### invalid number of `children`: `-1` and `20`. Replace by `1` and `2`.
Since `1` and `2` are the 2nd and 3rd frequent values, assumption is that there was a typo...

<span style="color:green"> Why do you think they are completely ivalid? Maybe it's a typo. Please explain </span>

In [ ]:
print(credit_scoring['children'].value_counts())
def fix_typo_children(children):
    if children == -1:
        return 1
    elif children == 20:
        return 2
    else:
        return children
credit_scoring['children'] = credit_scoring['children'].apply(fix_typo_children)

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


### Processing missing values

#### There are only 3445 out of 21,525 records with valid working days (non-negative `days_employed`). In addition, the median and mean of the positive values is ~365,000 - i.e 100 years, which points an error this data element.
As there is no way currently to fix the data, the column will be dropped.

In [ ]:
print(len(credit_scoring[credit_scoring['days_employed']>0]))
print(credit_scoring[credit_scoring['days_employed']>0]['days_employed'].mean())
credit_scoring = credit_scoring.drop('days_employed',axis=1)

3445
365004.3099162686


<span style="color:green"> You should definitely take a closer look at this column. Since the majority of data has negative values - these are definitely not missing. </span>

<span style="color:green"> Investigate the negative values in days_employed and think about what it can be. Is it a typo? Not days? Definitely not missing values. Also, does your mean value seem real to you? Can you think of real distribution of the days_employed for bank's clients?</span>

<span style="color:green"> I would be grateful if you also explain what you did here. :) Why did you create a new data set? Are you gonna work with it later? </span>

<span style="color:purple"> In general, droping columns is not a good practice. You never know what you are gonna need </span>

#### There are 54 duplicates, all with with missing (NaN) `total_income`. meaning, duplications probably due to total data information dropped during the data processing.

In [ ]:
duplicates = credit_scoring[credit_scoring.duplicated()]
print('Duplicated rows: {}'.format(len(duplicates)))
print('Rows missing total_income : {}'.format(len((duplicates[duplicates['total_income'].isna()]))))

Duplicated rows: 54
Rows missing total_income : 54


<span style="color:green"> Can you please explain why you think that? </span>

#### Replace `NaN` values in the column `total_income` by the mean, and convert column to integer.

In [ ]:
income_mean = credit_scoring['total_income'].mean()
print('Mean income: {:,.0f}'.format(income_mean))
credit_scoring['total_income'].fillna(income_mean, inplace=True)
credit_scoring['total_income'] = credit_scoring['total_income'].astype('int')

Mean income: 167,422


<span style="color:green"> Agree with mean for income :) </span>

### Conclusion

Missing values were found for both `days_employed` and `total_income` columns. The main issue was in `days_employed` which contained a lot of invaild values (negative) - as it seems this column is totaly corrupted and it is recommended that we recollect the information. Meanwhile the column `days_employed` is dropped. For `total_income` it was replaced by the mean.

<span style="color:green"> Unfortunately, in the real life we have to deal with this kind of information almost every day.So we have to work with what's given :)</span>

### Data type replacement

The `total_income` column was modified from float to integer - this will enable more consize storage and faster comptation.

<span style="color:green"> Since you have already done data replacement, here you only needed to leave a note about that :) </span>

### Conclusion

### Processing duplicates

#### There is duplicate data due to case differences in variable education (ex. secondary education and Secondary Education. Therefore `education` will be converted to lower case. 

In [ ]:
print(credit_scoring.groupby('education_id')['education'].value_counts())
credit_scoring['education'] = credit_scoring['education'].str.lower()
print(credit_scoring.groupby('education_id')['education'].value_counts())

education_id  education          
0             masters degree          4718
              MASTERS DEGREE           274
              Masters Degree           268
1             secondary education    13750
              SECONDARY EDUCATION      772
              Secondary Education      711
2             bachelor degree          668
              Bachelor Degree           47
              BACHELOR DEGREE           29
3             primary education        250
              PRIMARY EDUCATION         17
              Primary Education         15
4             academic degree            4
              ACADEMIC DEGREE            1
              Academic Degree            1
Name: education, dtype: int64
education_id  education          
0             masters degree          5260
1             secondary education    15233
2             bachelor degree          744
3             primary education        282
4             academic degree            6
Name: education, dtype: int64


<span style="color:green"> Good methods! But in real life please try to do convertion to one case BEFORE finding duplicates. Convertion can lead to more duplicates, that you won't be able to catch unless you check again </span>

#### Drop duplicates

In [ ]:
credit_scoring.drop_duplicates(inplace=True)

### Conclusion

The reason for duplicated rows is that either `days_employed` or `total_income` are NaN - this caused the redundancy.

### Lemmatization

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemma = WordNetLemmatizer()

def lemmatize_text(text):
    words = nltk.word_tokenize(text)
    return [wordnet_lemma.lemmatize(w, pos = 'n') for w in words]

credit_scoring['purpose_lemmatized'] = credit_scoring['purpose'].apply(lemmatize_text)
print(credit_scoring['purpose_lemmatized'].value_counts())
print(len(credit_scoring['purpose_lemmatized'].value_counts()))

[car]                                                  972
[wedding, ceremony]                                    791
[having, a, wedding]                                   768
[to, have, a, wedding]                                 765
[real, estate, transaction]                            675
[buy, commercial, real, estate]                        661
[housing, transaction]                                 652
[buying, property, for, renting, out]                  651
[transaction, with, the, residential, real, estate]    650
[purchase, of, the, house]                             646
[housing]                                              646
[purchase, of, the, house, for, my, family]            638
[construction, of, own, property]                      635
[property]                                             633
[transaction, with, my, real, estate]                  627
[building, a, real, estate]                            624
[buy, real, estate]                                    6

### Conclusion

As can be noticed in the distribution above, reasons are repeated in different manners (ex. [to, get, asupplementary, education], [to, become, educated])
Lemmatization is required in order to create keywords which are necessary for categorization of the purpose of the loan. The categorization by itself was found as not enough since multiple keywords were raiseed for same reason:
For house additional keywords: 'housing','estate' and 'property'
For education additional keywords: 'educated' and 'university'

<span style="color:green"> Please, show the lemmatized column and describe the distribution a little bit. </span>

### Categorizing Data

In [ ]:
def get_reason(tokens):
    token_s = set(tokens)
    if token_s.intersection({'house','housing','estate','property'}):
        return 'house'
    elif token_s.intersection({'education', 'educated', 'university'}):
        return 'education'
    elif token_s.intersection({'car'}):
        return 'car'
    elif token_s.intersection({'wedding'}):
        return 'wedding'
    else:
        print (tokens)
        return error
credit_scoring['reason'] = credit_scoring['purpose_lemmatized'].apply(get_reason)
print(credit_scoring[['purpose', 'purpose_lemmatized', 'reason']])

                       purpose          purpose_lemmatized     reason
0        purchase of the house  [purchase, of, the, house]      house
1                 car purchase             [car, purchase]        car
2        purchase of the house  [purchase, of, the, house]      house
3      supplementary education  [supplementary, education]  education
4            to have a wedding      [to, have, a, wedding]    wedding
...                        ...                         ...        ...
21520     housing transactions      [housing, transaction]      house
21521        purchase of a car      [purchase, of, a, car]        car
21522                 property                  [property]      house
21523        buying my own car      [buying, my, own, car]        car
21524             to buy a car           [to, buy, a, car]        car

[21454 rows x 3 columns]


In [ ]:
pd.options.display.float_format = '{:,.0f}'.format
print(credit_scoring['total_income'].describe())
def income_level(income):
    if income < 107655:
        return 'Low(<107K)'
    elif income < 156106:
        return 'Mid(107K-156K)'
    elif income < 195752:
        return 'High(157K-195K)'
    else:
        return 'Highest(>195K)'

credit_scoring['income_level'] = credit_scoring['total_income'].apply(income_level)
print(credit_scoring['income_level'].value_counts())

count      21,454
mean      167,422
std        97,794
min        20,667
25%       107,623
50%       156,044
75%       195,813
max     2,265,604
Name: total_income, dtype: float64
Mid(107K-156K)     5368
Highest(>195K)     5368
Low(<107K)         5368
High(157K-195K)    5350
Name: income_level, dtype: int64


Create 3 groups Young (<30), Mid (30-50), Old (>50)

In [ ]:
def age_group(age):
    if age < 30:
        return 'Young(<30)'
    elif age <= 50:
        return 'Mid(30-50)'
    else:
        return 'Old(>50)'

credit_scoring['age_group'] = credit_scoring['dob_years'].apply(age_group)
print(credit_scoring['age_group'].value_counts())

Mid(30-50)    11630
Old(>50)       6644
Young(<30)     3180
Name: age_group, dtype: int64


### Conclusion

A new category `reason` was created based on the `purpose` column and its lemmatization. We had to apply a function in order to handle the multiple synonims detected in the lemmatization process (see above in lemmatization section). Thanks to this grouping, the number of groups was reduced from 37 (after lemmatization) to just 4 (house, education, car, wedding).

A new category `income_level` was created based on the `total_income` columns - this will enable looking for correlation between income level and repaying loan on time. The category includes 4 equaly sized groups: Low, Mid, Hig, Highest.

A new category `age_group` was created based on the `dob_years` column - it contains 3 groups, and has can be noticed the 'Mid(30-50) contains most of the data.

<span style="color:green"> Cool categorization! Please take a deeper look into groups and describe them a little :) </span>

### Step 3. Answer these questions

- Is there a relation between having kids and repaying a loan on time?

In [ ]:
data_pivot = credit_scoring.pivot_table(index=['children'], columns='debt', values='education', aggfunc='count')
data_pivot.fillna(0, inplace=True)
data_pivot['%Return'] = 100*data_pivot[0] / (data_pivot[0] + data_pivot[1])
print(data_pivot.sort_values(by='%Return', ascending=False))

debt          0     1  %Return
children                      
5             9     0      100
0        13,028 1,063       92
3           303    27       92
1         4,410   445       91
2         1,926   202       91
4            37     4       90


### Conclusion

People with no or 3 children tend more to repay loan on time. However the difference is marginal (~1%)

<span style="color:green"> What about people with 3 children?:) But yeah, I agree with your methods) </span>

- Is there a relation between marital status and repaying a loan on time?

In [ ]:
data_pivot = credit_scoring.pivot_table(index=['family_status'], columns='debt', values='reason', aggfunc='count')
data_pivot['%Return'] = 100*data_pivot[0] / (data_pivot[0] + data_pivot[1])
print(data_pivot.sort_values(by='%Return', ascending=False))

debt                   0    1  %Return
family_status                         
widow / widower      896   63       93
divorced            1110   85       93
married            11408  931       92
civil partnership   3763  388       91
unmarried           2536  274       90


### Conclusion

Married people tend more to repay loan on time. However the difference is less than 2%

- Is there a relation between income level and repaying a loan on time?

In [ ]:
data_pivot = credit_scoring.pivot_table(index=['income_level'], columns='debt', values='education', aggfunc='count')
data_pivot.fillna(0, inplace=True)
data_pivot['%Return'] = 100*data_pivot[0] / (data_pivot[0] + data_pivot[1])
print(data_pivot.sort_values(by='%Return', ascending=False))

debt                0    1  %Return
income_level                       
Highest(>195K)   4985  383       93
Low(<107K)       4941  427       92
High(157K-195K)  4888  462       91
Mid(107K-156K)   4899  469       91


### Conclusion

people with the highest (25%) income tend more to repay loan on time. However the difference is less than 2%

- How do different loan purposes affect on-time repayment of the loan?

In [ ]:
data_pivot = credit_scoring.pivot_table(index=['reason'], columns='debt', values='education', aggfunc='count')
data_pivot.fillna(0, inplace=True)
data_pivot['%Return'] = 100*data_pivot[0] / (data_pivot[0] + data_pivot[1])
print(data_pivot.sort_values(by='%Return', ascending=False))

debt           0    1  %Return
reason                        
house      10029  782       93
wedding     2138  186       92
education   3643  370       91
car         3903  403       91


### Conclusion

people taking loan for buying a house tend more to repay loan on time. However the difference is quite small.

## Pivot tables for additional columns  we also generated:

In [ ]:
data_pivot = credit_scoring.pivot_table(index=['income_type'], columns='debt', values='education', aggfunc='count')
data_pivot.fillna(0, inplace=True)
data_pivot['%Return'] = 100*data_pivot[0] / (data_pivot[0] + data_pivot[1])
print(data_pivot.sort_values(by='%Return', ascending=False))

debt                             0     1  %Return
income_type                                      
entrepreneur                     2     0      100
student                          1     0      100
retiree                      3,613   216       94
civil servant                1,371    86       94
partner                      4,702   376       93
employee                    10,023 1,061       90
unempoyed                        1     1       50
paternity / maternity leave      0     1        0


In [ ]:
data_pivot = credit_scoring.pivot_table(index=['gender'], columns='debt', values='education', aggfunc='count')
data_pivot.fillna(0, inplace=True)
data_pivot['%Return'] = 100*data_pivot[0] / (data_pivot[0] + data_pivot[1])
print(data_pivot.sort_values(by='%Return', ascending=False))

debt        0    1  %Return
gender                     
F       13181  994       93
M        6532  747       90


In [ ]:
data_pivot = credit_scoring.pivot_table(index=['age_group'], columns='debt', values='education', aggfunc='count')
data_pivot.fillna(0, inplace=True)
data_pivot['%Return'] = 100*data_pivot[0] / (data_pivot[0] + data_pivot[1])
print(data_pivot.sort_values(by='%Return', ascending=False))

debt            0     1  %Return
age_group                       
Old(>50)     6263   381       94
Mid(30-50)  10619  1011       91
Young(<30)   2831   349       89


In [ ]:
data_pivot = credit_scoring.pivot_table(index=['education'], columns='debt', values='reason', aggfunc='count')
data_pivot.fillna(0, inplace=True)
data_pivot['%Return'] = 100*data_pivot[0] / (data_pivot[0] + data_pivot[1])
print(data_pivot.sort_values(by='%Return', ascending=False))

debt                     0     1  %Return
education                                
academic degree          6     0      100
masters degree       4,972   278       95
secondary education 13,808 1,364       91
bachelor degree        676    68       91
primary education      251    31       89


### Step 4. General conclusion

#### Main point notice - the whole purpose of the data is to predict return of loan. However the data contain only 1741 (8%) of total of people who DID NOT return loan. It is highly recommended to provide additional records of people who did not returned loan.
The rest of the data consisted of the following:
1. Family status - did not showed as generating impact on loan return
2. numer of children - no real impact on result
3. income level - there was missing data - recommended to retrieve it. was devided into 4 income groups - no real impact
4. reason of loan - was converted into new catregory with 4 groups
6. income type - no real impact
7. gender - it could be noticed female more reliable in returning loan.
8. education - the higher the education, higher chance to return loan
9. employment duration - information completely corrupted.

It is difficult to draw sharp conclusion as the differences are quite small. We should consider collecting additional parameters such as loan size, duration, area/region and crminal record.

<span style="color:green">Your general conlusion should be more about data in general. Please describe your data, your main points of your research and what you've found. Also, what bank should do to build a good loaning strategy based on your research? </span>

### Project Readiness Checklist

Put 'x' in the completed points. Then press Shift + Enter.

- [x]  file open;
- [x]  file examined;
- [x]  missing values defined;
- [x]  missing values are filled;
- [x]  an explanation of which missing value types were detected;
- [x]  explanation for the possible causes of missing values;
- [x]  an explanation of how the blanks are filled;
- [x]  replaced the real data type with an integer;
- [x]  an explanation of which method is used to change the data type and why;
- [x]  duplicates deleted;
- [x]  an explanation of which method is used to find and remove duplicates;
- [x]  description of the possible reasons for the appearance of duplicates in the data;
- [x]  highlighted lemmas in the values of the loan purpose column;
- [x]  the lemmatization process is described;
- [x]  data is categorized;
- [x]  an explanation of the principle of data categorization;
- [x]  an answer to the question "Is there a relation between having kids and repaying a loan on time?";
- [x]  an answer to the question " Is there a relation between marital status and repaying a loan on time?";
- [x]   an answer to the question " Is there a relation between income level and repaying a loan on time?";
- [x]  an answer to the question " How do different loan purposes affect on-time repayment of the loan?"
- [x]  conclusions are present on each stage;
- [x]  a general conclusion is made.